# 4 Wrangling - Combining SF Data<a id='4_Wrangling_-_Combining_SF_Data'></a>

## 4.1 Contents<a id='4.1_Contents'></a>
* [4 Wrangling - Combining SF Data](#4_Wrangling_-_Combining_SF_Data)
  * [4.1 Contents](#4.1_Contents)
  * [4.2 Introduction](#4.2_Introduction)
  * [4.3 Imports](#4.3_Imports)
  * [4.4 Load The Data](#4.4_Load_The_Data)
  * [4.5 Prepare data for merging](#4.5_Prepare_data_for_merging)
      * [4.5.1 Police Incident data](#4.5.1_Police_incident_data)
      * [4.5.2 311 Case data](#4.5.2_311_Case_data)
      * [4.5.3 Housing sales data](#4.5.3_Housing_sales_data)
  * [4.6 Merge The Data](#4.6_Merge_The_Data)
  * [4.7 Save data](#4.7_Save_data)

## 4.2 Introduction<a id='4.2_Introduction'></a>

In this notebook, we will combine all the data and prepare it for feature analysis and modeling.

Keep in mind that the objective here is to combine the datasets we have such that we can generate a model for predicting *housing sales price*.

At the end of this notebook, we will generate the following file:

  * SF_Combined_SFPD_311_Housing.csv : all SF police incident reports, 311 cases, and housing sales data aggregated by month and by neighborhood, from January 2018 up to and including September 2020, wherein each row is an observation with a distinct pairing on month-year and each column represents a possible feature to be used in modelling

## 4.3 Imports<a id='4.3_Imports'></a>

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

## 4.4 Load The Data<a id='4.4_Load_The_Data'></a>

In [2]:
df_sfpd = pd.read_csv('data/SF_Police_Incidents_aggregated.csv')

In [3]:
df_311 = pd.read_csv('data/311_Cases_aggregated.csv')

In [4]:
df_sales = pd.read_csv('data/Redfin_SF_sales_adjusted_neighborhood.csv')

In [5]:
print('sfpd incidents shape: ')
print(df_sfpd.shape)
print('311 cases shape: ')
print(df_311.shape)
print('housing sales shape: ')
print(df_sales.shape)

sfpd incidents shape: 
(157056, 7)
311 cases shape: 
(374480, 9)
housing sales shape: 
(3359, 9)


In [6]:
df_sfpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157056 entries, 0 to 157055
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Incident Year Month      157056 non-null  int64 
 1   Analysis Neighborhood    157056 non-null  object
 2   Report Type Description  157056 non-null  object
 3   Incident Category        157056 non-null  object
 4   Incident Description     157056 non-null  object
 5   Police District          157056 non-null  object
 6   Incident Count           157056 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 8.4+ MB


In [7]:
df_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374480 entries, 0 to 374479
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Opened Year Month  374480 non-null  int64 
 1   Source             374480 non-null  object
 2   Neighborhood       374480 non-null  object
 3   Police District    374480 non-null  object
 4   Status             374480 non-null  object
 5   Category           374480 non-null  object
 6   Request Type       374480 non-null  object
 7   Has Media          374480 non-null  bool  
 8   Case Count         374480 non-null  int64 
dtypes: bool(1), int64(2), object(6)
memory usage: 23.2+ MB


In [8]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3359 entries, 0 to 3358
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Sales Year Month          3359 non-null   int64  
 1   311 Neighborhood          3359 non-null   object 
 2   Analysis Neighborhood     3359 non-null   object 
 3   Median Sale Price         3359 non-null   float64
 4   Homes Sold                3359 non-null   int64  
 5   New Listings              3359 non-null   float64
 6   Inventory                 3359 non-null   float64
 7   Days on Market            3359 non-null   float64
 8   Average Sale To List Pct  3359 non-null   float64
dtypes: float64(5), int64(2), object(2)
memory usage: 236.3+ KB


**Note: `df_sfpd`'s `Analysis Neighborhood` will match to `df_sales`'s `Analysis Neighborhood`. `df_311`'s `Neighborhood` will match to `df_sales`'s `311 Neighborhood`.**

In order to properly map `df_sfpd` with `df_311`, we need to match up their Neighborhoods.

In [9]:
neighborhood_mapper = pd.read_csv('data/Neighborhoods_Map.csv')

In [10]:
neighborhood_mapper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   RedfinNeighborhood     130 non-null    object
 1   311 Neighborhood       130 non-null    object
 2   Analysis Neighborhood  130 non-null    object
dtypes: object(3)
memory usage: 3.2+ KB


## 4.5 Prepare data for merging<a id='4.5_Prepare_data_for_merging'></a>

Before we merge the data, let's think about what we want the end result to look like. Our objective here is to create a model using the dataset, and to do that, we will need to performing regression analysis. Thus, every record (row) should be an observation. In this case, since we are planning on performing an analysis over time for every neighborhood, every record will be its own distinct month-year, neighborhood pairing. Every observation will share the same set of features (columns). Thus, the dataset will be in a wide format.

Before we merge the data, we need to arrange each data frame accordingly.

### 4.5.1 Police Incident data<a id='4.5.1_Police_incident_data'></a>

In [11]:
df_sfpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157056 entries, 0 to 157055
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Incident Year Month      157056 non-null  int64 
 1   Analysis Neighborhood    157056 non-null  object
 2   Report Type Description  157056 non-null  object
 3   Incident Category        157056 non-null  object
 4   Incident Description     157056 non-null  object
 5   Police District          157056 non-null  object
 6   Incident Count           157056 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 8.4+ MB


In [12]:
# let's rename a few columns for consistency
df_sfpd.rename(columns={'Incident Year Month':'Year Month', 'Analysis Neighborhood':'Neighborhood'}, inplace=True)

In [13]:
df_sfpd.head()

,Year Month,Neighborhood,Report Type Description,Incident Category,Incident Description,Police District,Incident Count
0,201801,Bayview Hunters Point,Coplogic Initial,Larceny Theft,"Theft, From Locked Vehicle, >$950",Bayview,24
1,201801,Bayview Hunters Point,Coplogic Initial,Larceny Theft,"Theft, From Unlocked Vehicle, >$950",Bayview,3
2,201801,Bayview Hunters Point,Coplogic Initial,Larceny Theft,"Theft, Other Property, $50-$200",Bayview,4
3,201801,Bayview Hunters Point,Coplogic Initial,Larceny Theft,"Theft, Other Property, $50-$200",Mission,1
4,201801,Bayview Hunters Point,Coplogic Initial,Larceny Theft,"Theft, Other Property, >$950",Bayview,3


In [14]:
# use this as a check
df_sfpd[(df_sfpd['Year Month']==201801) 
            & (df_sfpd['Neighborhood']=='Bayview Hunters Point')].groupby(['Report Type Description','Incident Category'])['Incident Count'].sum()

Report Type Description  Incident Category  
Coplogic Initial         Larceny Theft          35
                         Lost Property           3
                         Malicious Mischief     11
Coplogic Supplement      Burglary                2
                         Larceny Theft          10
                                                ..
Vehicle Initial          Robbery                 1
                         Stolen Property         1
Vehicle Supplement       Motor Vehicle Theft     3
                         Other Offenses          1
                         Recovered Vehicle      33
Name: Incident Count, Length: 67, dtype: int64

In [15]:
# testing the pivot
df_sfpd[(df_sfpd['Year Month']==201801) 
            & (df_sfpd['Neighborhood']=='Bayview Hunters Point')].pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Report Type Description','Incident Category'],
                                values='Incident Count',
                                aggfunc='sum',
                                fill_value=0)

Report Type Description          Coplogic Initial                \
Incident Category                   Larceny Theft Lost Property   
Year Month Neighborhood                                           
201801     Bayview Hunters Point               35             3   

Report Type Description                             Coplogic Supplement  \
Incident Category                Malicious Mischief            Burglary   
Year Month Neighborhood                                                   
201801     Bayview Hunters Point                 11                   2   

Report Type Description                        Initial                   \
Incident Category                Larceny Theft   Arson Assault Burglary   
Year Month Neighborhood                                                   
201801     Bayview Hunters Point            10       5      59       29   

Report Type Description                                              ...  \
Incident Category                Courtesy Report Disorderly Conduct  ...   
Year Month Neighborhood                                              ...   
201801     Bayview Hunters Point               1                 25  ...   

Report Type Description             Vehicle Initial                      \
Incident Category                Malicious Mischief Motor Vehicle Theft   
Year Month Neighborhood                                                   
201801     Bayview Hunters Point                  1                  37   

Report Type Description                                              \
Incident Category                Other Miscellaneous Other Offenses   
Year Month Neighborhood                                               
201801     Bayview Hunters Point                   1              3   

Report Type Description                                                     \
Incident Category                Recovered Vehicle Robbery Stolen Property   
Year Month Neighborhood                                                      
201801     Bayview Hunters Point                 3       1               1   

Report Type Description           Vehicle Supplement                 \
Incident Category                Motor Vehicle Theft Other Offenses   
Year Month Neighborhood                                               
201801     Bayview Hunters Point                   3              1   

Report Type Description                             
Incident Category                Recovered Vehicle  
Year Month Neighborhood                             
201801     Bayview Hunters Point                33  

[1 rows x 67 columns]

In order to reduce sparsity of the data, we can eliminate `Report Type Description` from the pivot and simply use the `Incident Category`.

In [65]:
# use this as a check
df_sfpd[(df_sfpd['Year Month']==201801) 
            & (df_sfpd['Neighborhood']=='Bayview Hunters Point')].groupby(['Incident Category'])['Incident Count'].sum()

Incident Category
Arson                                         5
Assault                                      64
Burglary                                     42
Case Closure                                  1
Courtesy Report                               1
Disorderly Conduct                           27
Drug Offense                                 16
Embezzlement                                  4
Family Offense                                2
Fire Report                                   1
Forgery And Counterfeiting                    3
Fraud                                        13
Larceny Theft                               105
Lost Property                                11
Malicious Mischief                           60
Miscellaneous Investigation                  10
Missing Person                               19
Motor Vehicle Theft                          44
Non-Criminal                                 51
Offences Against The Family And Children     16
Other                 

In [66]:
# testing the pivot
df_sfpd[(df_sfpd['Year Month']==201801) 
            & (df_sfpd['Neighborhood']=='Bayview Hunters Point')].pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Incident Category'],
                                values='Incident Count',
                                aggfunc='sum',
                                fill_value=0)

,Incident Category,Arson,Assault,Burglary,Case Closure,Courtesy Report,Disorderly Conduct,Drug Offense,Embezzlement,Family Offense,Fire Report,...,Recovered Vehicle,Robbery,Sex Offense,Stolen Property,Suspicious Occ,Traffic Collision,Traffic Violation Arrest,Warrant,Weapons Carrying Etc,Weapons Offense
Year Month,Neighborhood,,,,,,,,,,,,,,,,,,,,,
201801,Bayview Hunters Point,5,64,42,1,1,27,16,4,2,1,...,36,25,1,7,19,5,12,25,22,8


Looks good, we have halved the number of features!

In [67]:
# create the table
df_sfpd_table = df_sfpd.pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Incident Category'],
                                values='Incident Count',
                                aggfunc='sum',
                                fill_value=0)

In [68]:
df_sfpd_table.shape

(1353, 50)

In [69]:
df_sfpd_table.head()

Incident Category                 Arson  Assault  Burglary  Case Closure  \
Year Month Neighborhood                                                    
201801     Bayview Hunters Point      5       64        42             1   
           Bernal Heights             2       22         8             1   
           Castro/Upper Market        1       19        29             1   
           Chinatown                  1       12        10             1   
           Excelsior                  0       24        12             0   

Incident Category                 Civil Sidewalks  Courtesy Report  \
Year Month Neighborhood                                              
201801     Bayview Hunters Point                0                1   
           Bernal Heights                       0                0   
           Castro/Upper Market                  3                0   
           Chinatown                            0               10   
           Excelsior                            0                0   

Incident Category                 Disorderly Conduct  Drug Offense  \
Year Month Neighborhood                                              
201801     Bayview Hunters Point                  27            16   
           Bernal Heights                          8             0   
           Castro/Upper Market                     2             5   
           Chinatown                               2             3   
           Excelsior                               5             6   

Incident Category                 Drug Violation  Embezzlement  ...  \
Year Month Neighborhood                                         ...   
201801     Bayview Hunters Point               0             4  ...   
           Bernal Heights                      0             0  ...   
           Castro/Upper Market                 0             0  ...   
           Chinatown                           0             2  ...   
           Excelsior                           0             0  ...   

Incident Category                 Suspicious Occ  Traffic Collision  \
Year Month Neighborhood                                               
201801     Bayview Hunters Point              19                  5   
           Bernal Heights                      5                  0   
           Castro/Upper Market                 8                  1   
           Chinatown                           9                  0   
           Excelsior                           4                  0   

Incident Category                 Traffic Violation Arrest  Vandalism  \
Year Month Neighborhood                                                 
201801     Bayview Hunters Point                        12          0   
           Bernal Heights                                4          0   
           Castro/Upper Market                           2          0   
           Chinatown                                     3          0   
           Excelsior                                     6          1   

Incident Category                 Vehicle Impounded  Vehicle Misplaced  \
Year Month Neighborhood                                                  
201801     Bayview Hunters Point                  0                  0   
           Bernal Heights                         0                  0   
           Castro/Upper Market                    0                  0   
           Chinatown                              0                  0   
           Excelsior                              0                  0   

Incident Category                 Warrant  Weapons Carrying Etc  \
Year Month Neighborhood                                           
201801     Bayview Hunters Point       25                    22   
           Bernal Heights               7                     0   
           Castro/Upper Market         19                     0   
           Chinatown                    2                     2   
           Excelsior                    5               

In [70]:
df_sfpd_table.describe()

Incident Category,Arson,Assault,Burglary,Case Closure,Civil Sidewalks,Courtesy Report,Disorderly Conduct,Drug Offense,Drug Violation,Embezzlement,...,Suspicious Occ,Traffic Collision,Traffic Violation Arrest,Vandalism,Vehicle Impounded,Vehicle Misplaced,Warrant,Weapons Carrying Etc,Weapons Offence,Weapons Offense
count,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,...,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000
mean,0.736881,16.966001,14.386548,1.203991,0.634885,0.826312,4.966741,7.176644,0.089431,0.406504,...,5.524760,0.554324,3.478936,0.447894,0.198078,0.124169,9.838877,1.475979,0.012565,1.580192
std,1.388437,25.721547,15.374632,2.476472,3.715038,1.832115,6.662594,23.262758,0.407201,0.977719,...,6.560225,1.029971,6.584883,0.834082,0.655493,0.400757,19.448743,3.177150,0.123994,2.777705
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,3.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,0.000000,8.000000,10.000000,0.000000,0.000000,0.000000,3.000000,1.000000,0.000000,0.000000,...,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,1.000000
75%,1.000000,15.000000,19.000000,1.000000,0.000000,1.000000,6.000000,4.000000,0.000000,0.000000,...,7.000000,1.000000,4.000000,1.000000,0.000000,0.000000,8.000000,1.000000,0.000000,2.000000
max,10.000000,138.000000,106.000000,26.000000,72.000000,18.000000,42.000000,225.000000,5.000000,8.000000,...,45.000000,8.000000,59.000000,6.000000,7.000000,6.000000,129.000000,27.000000,2.000000,18.000000


In [80]:
df_sfpd_table.columns

Index(['Arson', 'Assault', 'Burglary', 'Case Closure', 'Civil Sidewalks',
       'Courtesy Report', 'Disorderly Conduct', 'Drug Offense',
       'Drug Violation', 'Embezzlement', 'Family Offense', 'Fire Report',
       'Forgery And Counterfeiting', 'Fraud', 'Gambling', 'Homicide',
       'Human Trafficking (A), Commercial Sex Acts',
       'Human Trafficking (B), Involuntary Servitude',
       'Human Trafficking, Commercial Sex Acts', 'Larceny Theft',
       'Liquor Laws', 'Lost Property', 'Malicious Mischief',
       'Miscellaneous Investigation', 'Missing Person', 'Motor Vehicle Theft',
       'Motor Vehicle Theft?', 'Non-Criminal',
       'Offences Against The Family And Children', 'Other',
       'Other Miscellaneous', 'Other Offenses', 'Prostitution', 'Rape',
       'Recovered Vehicle', 'Robbery', 'Sex Offense', 'Stolen Property',
       'Suicide', 'Suspicious', 'Suspicious Occ', 'Traffic Collision',
       'Traffic Violation Arrest', 'Vandalism', 'Vehicle Impounded',
       'Vehi

### 4.5.2 311 Case data<a id='4.5.2_311_Case_data'></a>

In [23]:
df_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374480 entries, 0 to 374479
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Opened Year Month  374480 non-null  int64 
 1   Source             374480 non-null  object
 2   Neighborhood       374480 non-null  object
 3   Police District    374480 non-null  object
 4   Status             374480 non-null  object
 5   Category           374480 non-null  object
 6   Request Type       374480 non-null  object
 7   Has Media          374480 non-null  bool  
 8   Case Count         374480 non-null  int64 
dtypes: bool(1), int64(2), object(6)
memory usage: 23.2+ MB


In [24]:
# join with neighborhood mapper to get the Analysis Neighborhood

# fetch distinct 311 neighborhoods
neighborhood_mapper_311 = neighborhood_mapper[['311 Neighborhood','Analysis Neighborhood']].drop_duplicates()

In [25]:
df_311_neighborhood = pd.merge( df_311, neighborhood_mapper_311, left_on='Neighborhood', right_on='311 Neighborhood', validate='many_to_one')

In [26]:
df_311_neighborhood.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374480 entries, 0 to 374479
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Opened Year Month      374480 non-null  int64 
 1   Source                 374480 non-null  object
 2   Neighborhood           374480 non-null  object
 3   Police District        374480 non-null  object
 4   Status                 374480 non-null  object
 5   Category               374480 non-null  object
 6   Request Type           374480 non-null  object
 7   Has Media              374480 non-null  bool  
 8   Case Count             374480 non-null  int64 
 9   311 Neighborhood       374480 non-null  object
 10  Analysis Neighborhood  374480 non-null  object
dtypes: bool(1), int64(2), object(8)
memory usage: 31.8+ MB


In [27]:
# drop unneeded columns
df_311_neighborhood.drop(columns=['Neighborhood','311 Neighborhood'], inplace=True)

In [28]:
# let's rename a few columns for consistency
df_311_neighborhood.rename(columns={'Opened Year Month':'Year Month', 'Analysis Neighborhood':'Neighborhood'}, inplace=True)

In [29]:
df_311_neighborhood.head()

,Year Month,Source,Police District,Status,Category,Request Type,Has Media,Case Count,Neighborhood
0,201801,Integrated Agency,NORTHERN,Closed,Graffiti,Graffiti on Building,False,2,Hayes Valley
1,201801,Integrated Agency,NORTHERN,Closed,Street Defects,Pavement_Defect,False,1,Hayes Valley
2,201801,Integrated Agency,NORTHERN,Closed,Street and Sidewalk Cleaning,Bulky Items,False,2,Hayes Valley
3,201801,Mobile/Open311,NORTHERN,Closed,Blocked Street or SideWalk,Blocked_Sidewalk,False,1,Hayes Valley
4,201801,Mobile/Open311,NORTHERN,Closed,Damaged Property,Damaged Fire_Police_Callbox,True,1,Hayes Valley


In [30]:
# use this as a check
df_311_neighborhood[(df_311_neighborhood['Year Month']==201801) 
                    & (df_311_neighborhood['Neighborhood']=='Hayes Valley')
                    & (df_311_neighborhood['Category']=='Graffiti')].groupby(['Source','Status','Has Media'])['Case Count'].sum()

Source             Status  Has Media
Integrated Agency  Closed  False          4
Mobile/Open311     Closed  False         11
                           True         183
Phone              Closed  False         12
Web                Closed  False         23
Name: Case Count, dtype: int64

In [31]:
# testing the pivot
df_311_neighborhood[(df_311_neighborhood['Year Month']==201801) 
                    & (df_311_neighborhood['Neighborhood']=='Hayes Valley')
                    & (df_311_neighborhood['Category']=='Graffiti')].pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Source','Status','Category','Has Media'],
                                values='Case Count',
                                aggfunc='sum',
                                fill_value=0)

Source                  Integrated Agency Mobile/Open311          Phone  \
Status                             Closed         Closed         Closed   
Category                         Graffiti       Graffiti       Graffiti   
Has Media                           False          False True     False   
Year Month Neighborhood                                                   
201801     Hayes Valley                 4             11   183       12   

Source                       Web  
Status                    Closed  
Category                Graffiti  
Has Media                  False  
Year Month Neighborhood           
201801     Hayes Valley       23

In [72]:
# if we pivot in this way, what would be the shape of the table
df_311_neighborhood.pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Source','Status','Category','Has Media'],
                                values='Case Count',
                                aggfunc='sum',
                                fill_value=0).shape

(1353, 368)

That's a large number of columns, which will contribute to a sparse dataset. 

As we did for the police incident data, let's think about the usefulness of the columns we are including in the pivot.
  * `Status`: The values here will either be Open or Closed. However, more recent cases are more likely to be Open, simply due to the timing of data retrieval. This could lead to bias. Better to remove this.
  * `Has Media`: Submissions via Mobile/Open311 (and Web and Email, to a lesser extent) are more likely to include photos. They are fairly unlikely to come from submissions via Phone or Integrated Agency. In the context of this problem, photos will be submitted for certain types of request (Sidewalk Trash, Parking Violations) than they are for others, which means there may not be much value added from this. Let's remove this as well.
  * `Source`: For the same reason that we removed the Police Incident data's Report Type Description, we should remove this.

In [73]:
# use this as a check
df_311_neighborhood[(df_311_neighborhood['Year Month']==201801) 
                    & (df_311_neighborhood['Neighborhood']=='Hayes Valley')].groupby(['Category'])['Case Count'].sum()

Category
311 External Request              2
Abandoned Vehicle                 2
Blocked Street or SideWalk       22
Catch Basin Maintenance           2
Color Curb                        1
Damaged Property                 22
Encampments                      66
General Request                  42
Graffiti                        233
Homeless Concerns                34
Illegal Postings                  8
Litter Receptacles               18
MUNI Feedback                    16
Noise Report                     10
Rec and Park Requests            22
Residential Building Request      1
Sewer Issues                     14
Sidewalk or Curb                  5
Sign Repair                      21
Street Defects                   15
Street and Sidewalk Cleaning    478
Streetlights                      8
Temporary Sign Request            3
Tree Maintenance                 30
Name: Case Count, dtype: int64

In [74]:
# testing the pivot
df_311_neighborhood[(df_311_neighborhood['Year Month']==201801) 
                    & (df_311_neighborhood['Neighborhood']=='Hayes Valley')].pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Category'],
                                values='Case Count',
                                aggfunc='sum',
                                fill_value=0)

,Category,311 External Request,Abandoned Vehicle,Blocked Street or SideWalk,Catch Basin Maintenance,Color Curb,Damaged Property,Encampments,General Request,Graffiti,Homeless Concerns,...,Rec and Park Requests,Residential Building Request,Sewer Issues,Sidewalk or Curb,Sign Repair,Street Defects,Street and Sidewalk Cleaning,Streetlights,Temporary Sign Request,Tree Maintenance
Year Month,Neighborhood,,,,,,,,,,,,,,,,,,,,,
201801,Hayes Valley,2,2,22,2,1,22,66,42,233,34,...,22,1,14,5,21,15,478,8,3,30


Great, data looks much less sparse!

In [75]:
# create the table
df_311_table = df_311_neighborhood.pivot_table(index=['Year Month','Neighborhood'],
                                columns=['Category'],
                                values='Case Count',
                                aggfunc='sum',
                                fill_value=0)

In [76]:
df_311_table.shape

(1353, 29)

In [77]:
df_311_table.head()

Category                          311 External Request  Abandoned Vehicle  \
Year Month Neighborhood                                                     
201801     Bayview Hunters Point                     4                288   
           Bernal Heights                            2                257   
           Castro/Upper Market                       2                 27   
           Chinatown                                 0                  0   
           Excelsior                                 3                100   

Category                          Blocked Street or SideWalk  \
Year Month Neighborhood                                        
201801     Bayview Hunters Point                          10   
           Bernal Heights                                  8   
           Castro/Upper Market                             6   
           Chinatown                                       3   
           Excelsior                                       4   

Category                          Catch Basin Maintenance  Color Curb  \
Year Month Neighborhood                                                 
201801     Bayview Hunters Point                        2           3   
           Bernal Heights                               5           4   
           Castro/Upper Market                          2           2   
           Chinatown                                    0           0   
           Excelsior                                    1           0   

Category                          DPW Volunteer Programs  Damaged Property  \
Year Month Neighborhood                                                      
201801     Bayview Hunters Point                       3                21   
           Bernal Heights                              0                17   
           Castro/Upper Market                         0                27   
           Chinatown                                   0                15   
           Excelsior                                   0                 6   

Category                          Encampments  General Request  Graffiti  ...  \
Year Month Neighborhood                                                   ...   
201801     Bayview Hunters Point          122               93        67  ...   
           Bernal Heights                  71               62        77  ...   
           Castro/Upper Market            241               44       200  ...   
           Chinatown                       18               13       270  ...   
           Excelsior                        7               49        86  ...   

Category                          Residential Building Request  SFHA Requests  \
Year Month Neighborhood                                                         
201801     Bayview Hunters Point                             8             16   
           Bernal Heights                                    4              0   
           Castro/Upper Market                               1              0   
           Chinatown                                         0              2   
           Excelsior                                         3              0   

Category                          Sewer Issues  Sidewalk or Curb  Sign Repair  \
Year Month Neighborhood                                                         
201801     Bayview Hunters Point            81                13            5   
           Bernal Heights                   44                 8            9   
           Castro/Upper Market              26                17            8   
           Chinatown                         4                 9           20   
           Excelsior                        37                 3            5   

Category                          Street Defects  \
Year Month Neighborhood                            
201801     Bayview Hunters Point              39   
           Bernal Heights                     20   
           Castro/Upper Market          

In [78]:
df_311_table.describe()

Category,311 External Request,Abandoned Vehicle,Blocked Street or SideWalk,Catch Basin Maintenance,Color Curb,DPW Volunteer Programs,Damaged Property,Encampments,General Request,Graffiti,...,Residential Building Request,SFHA Requests,Sewer Issues,Sidewalk or Curb,Sign Repair,Street Defects,Street and Sidewalk Cleaning,Streetlights,Temporary Sign Request,Tree Maintenance
count,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,...,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000
mean,1.288987,49.236511,8.842572,1.118256,0.583888,0.163341,18.720621,129.683666,76.331855,124.045085,...,3.069475,2.914265,20.239468,14.962306,13.892831,15.507021,500.152254,9.090170,4.083518,21.580931
std,1.704503,71.298814,11.484794,1.766109,1.505102,0.806272,21.599029,282.649231,81.516851,193.808712,...,5.031812,11.780916,18.759542,18.190581,13.926621,22.203040,577.284665,10.128063,8.192017,20.148476
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000,5.000000,10.000000,25.000000,30.000000,...,0.000000,0.000000,7.000000,4.000000,5.000000,5.000000,142.000000,2.000000,0.000000,8.000000
50%,1.000000,20.000000,5.000000,0.000000,0.000000,0.000000,13.000000,28.000000,55.000000,74.000000,...,1.000000,0.000000,16.000000,10.000000,10.000000,11.000000,392.000000,6.000000,1.000000,18.000000
75%,2.000000,63.000000,11.000000,2.000000,0.000000,0.000000,22.000000,106.000000,99.000000,151.000000,...,4.000000,0.000000,27.000000,19.000000,18.000000,20.000000,643.000000,12.000000,5.000000,29.000000
max,15.000000,428.000000,106.000000,16.000000,12.000000,10.000000,131.000000,2210.000000,536.000000,2390.000000,...,37.000000,84.000000,150.000000,219.000000,117.000000,631.000000,4605.000000,67.000000,85.000000,193.000000


In [79]:
df_311_table.columns

Index(['311 External Request', 'Abandoned Vehicle',
       'Blocked Street or SideWalk', 'Catch Basin Maintenance', 'Color Curb',
       'DPW Volunteer Programs', 'Damaged Property', 'Encampments',
       'General Request', 'Graffiti', 'Homeless Concerns', 'Illegal Postings',
       'Litter Receptacles', 'MUNI Feedback', 'Muni Employee Feedback',
       'Muni Service Feedback', 'Noise Report', 'Parking Enforcement',
       'Rec and Park Requests', 'Residential Building Request',
       'SFHA Requests', 'Sewer Issues', 'Sidewalk or Curb', 'Sign Repair',
       'Street Defects', 'Street and Sidewalk Cleaning', 'Streetlights',
       'Temporary Sign Request', 'Tree Maintenance'],
      dtype='object', name='Category')

### 4.5.3 Housing sales data<a id='4.5.3_Housing_sales_data'></a>

In [39]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3359 entries, 0 to 3358
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Sales Year Month          3359 non-null   int64  
 1   311 Neighborhood          3359 non-null   object 
 2   Analysis Neighborhood     3359 non-null   object 
 3   Median Sale Price         3359 non-null   float64
 4   Homes Sold                3359 non-null   int64  
 5   New Listings              3359 non-null   float64
 6   Inventory                 3359 non-null   float64
 7   Days on Market            3359 non-null   float64
 8   Average Sale To List Pct  3359 non-null   float64
dtypes: float64(5), int64(2), object(2)
memory usage: 236.3+ KB


In [83]:
# note that since we are only interested in our target feature of 'Median Sale Price', that is the only column we need to keep
sales_cols_to_groupby = ['Sales Year Month', 'Analysis Neighborhood']
aggregated_sales = df_sales.groupby(sales_cols_to_groupby)['Median Sale Price'].mean()
aggregated_sales = aggregated_sales.reset_index()

In [84]:
aggregated_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1192 entries, 0 to 1191
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Sales Year Month       1192 non-null   int64  
 1   Analysis Neighborhood  1192 non-null   object 
 2   Median Sale Price      1192 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 28.1+ KB


In [85]:
# let's rename a few columns for consistency
aggregated_sales.rename(columns={'Sales Year Month':'Year Month', 'Analysis Neighborhood':'Neighborhood'}, inplace=True)

In [86]:
aggregated_sales.head()

,Year Month,Neighborhood,Median Sale Price
0,201801,Bayview Hunters Point,830000.0
1,201801,Bernal Heights,1538500.0
2,201801,Castro/Upper Market,1351000.0
3,201801,Chinatown,1500000.0
4,201801,Excelsior,985000.0


In [87]:
aggregated_sales.shape

(1192, 3)

**We notice that we have fewer rows in the sales (1192) than in the others (1353)**

In [88]:
# first, confirm there are no differences between sfpd and 311 , make use of the MultiIndex
print(df_sfpd_table.index.difference(df_311_table.index))
print(df_311_table.index.difference(df_sfpd_table.index))

MultiIndex([], names=['Year Month', 'Neighborhood'])
MultiIndex([], names=['Year Month', 'Neighborhood'])


In [89]:
# find the difference between 311 and sales
index_difference = df_311_table.index.difference(list(zip(aggregated_sales['Year Month'],aggregated_sales['Neighborhood'])))

In [90]:
index_difference

MultiIndex([(201801, 'Golden Gate Park'),
            (201801,     'Lincoln Park'),
            (201801,     'McLaren Park'),
            (201801,         'Presidio'),
            (201801,  'Treasure Island'),
            (201802, 'Golden Gate Park'),
            (201802,     'Lincoln Park'),
            (201802,     'McLaren Park'),
            (201802,         'Presidio'),
            (201802,  'Treasure Island'),
            ...
            (202008, 'Golden Gate Park'),
            (202008,     'Lincoln Park'),
            (202008,     'McLaren Park'),
            (202008,         'Presidio'),
            (202008,  'Treasure Island'),
            (202009, 'Golden Gate Park'),
            (202009,     'Lincoln Park'),
            (202009,     'McLaren Park'),
            (202009,         'Presidio'),
            (202009,  'Treasure Island')],
           names=['Year Month', 'Neighborhood'], length=161)

In [91]:
index_difference.get_level_values(1).unique()

Index(['Golden Gate Park', 'Lincoln Park', 'McLaren Park', 'Presidio',
       'Treasure Island', 'Chinatown'],
      dtype='object', name='Neighborhood')

It makes sense that there are no housing sales for the parks (Golden Gate Park, Lincoln Park, and McLaren Park) and for Treasure Island, so we can consider dropping that data in the police incident reports and the 311 cases, but let's make sure that the other regions (Presidio and Chinatown) do contain some housing sales data.

In [92]:
aggregated_sales[aggregated_sales['Neighborhood'].isin(index_difference.get_level_values(1).unique())]['Neighborhood'].value_counts()

Chinatown    26
Presidio     11
Name: Neighborhood, dtype: int64

Looks fine, so when we merge the data, we can just fill null values with 0.

## 4.6 Merge The Data<a id='4.6_Merge_The_Data'></a>

It's time to merge the data and see how it looks all together.

In [93]:
# remember that we have multi-indexes for sfpd and 311 but not for housing sales
combined_sfpd_311 = pd.merge(df_sfpd_table, df_311_table, left_index=True, right_index=True).reset_index()

In [94]:
combined_sfpd_311_sales = pd.merge(combined_sfpd_311, aggregated_sales, on=['Year Month','Neighborhood'], how='left').fillna(0)

In [95]:
combined_sfpd_311_sales.shape

(1353, 82)

In [96]:
combined_sfpd_311_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1353 entries, 0 to 1352
Data columns (total 82 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Year Month                                    1353 non-null   int64  
 1   Neighborhood                                  1353 non-null   object 
 2   Arson                                         1353 non-null   int64  
 3   Assault                                       1353 non-null   int64  
 4   Burglary                                      1353 non-null   int64  
 5   Case Closure                                  1353 non-null   int64  
 6   Civil Sidewalks                               1353 non-null   int64  
 7   Courtesy Report                               1353 non-null   int64  
 8   Disorderly Conduct                            1353 non-null   int64  
 9   Drug Offense                                  1353 non-null   i

In [59]:
combined_sfpd_311_sales.describe()

,Year Month,"(Coplogic Initial, Burglary)","(Coplogic Initial, Disorderly Conduct)","(Coplogic Initial, Forgery And Counterfeiting)","(Coplogic Initial, Fraud)","(Coplogic Initial, Larceny Theft)","(Coplogic Initial, Lost Property)","(Coplogic Initial, Malicious Mischief)","(Coplogic Supplement, Burglary)","(Coplogic Supplement, Forgery And Counterfeiting)",...,"(Web, Open, Streetlights, False)","(Web, Open, Streetlights, True)","(Web, Open, Tree Maintenance, False)","(Web, Open, Tree Maintenance, True)",Median Sale Price,Homes Sold,New Listings,Inventory,Days on Market,Average Sale To List Pct
count,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,...,1353.000000,1353.000000,1353.000000,1353.000000,1.353000e+03,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000
mean,201897.000000,0.028086,0.000739,0.062823,0.240207,34.543237,4.121212,4.361419,1.019217,0.011086,...,0.085735,0.028825,0.108647,0.092387,1.345813e+06,40.271249,55.450850,31.253511,27.573597,94.733398
std,78.786131,0.201571,0.027186,0.274212,0.821734,37.698741,6.513193,4.384469,1.710458,0.104746,...,0.326419,0.180146,0.407972,0.406540,7.621753e+05,47.317381,73.899475,55.998125,22.167308,35.316884
min,201801.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,201809.000000,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.006000e+06,11.000000,15.000000,6.000000,16.000000,101.050000
50%,201905.000000,0.000000,0.000000,0.000000,0.000000,22.000000,2.000000,3.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.375000e+06,28.000000,36.000000,16.000000,23.000000,105.925000
75%,202001.000000,0.000000,0.000000,0.000000,0.000000,47.000000,5.000000,6.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.725000e+06,51.000000,68.000000,34.000000,34.500000,110.650000
max,202009.000000,3.000000,1.000000,3.000000,13.000000,237.000000,49.000000,27.000000,20.000000,1.000000,...,3.000000,2.000000,4.000000,4.000000,7.925000e+06,311.000000,564.000000,606.000000,253.000000,129.333333


## 4.7 Save data<a id='4.7_Save_data'></a>

In [60]:
datapath = 'data'

# create datapath if it doesn't exist
if not os.path.exists(datapath):
    os.mkdir(datapath)

In [62]:
# write combined data
datapath_combineddata = os.path.join(datapath, 'SF_Combined_SFPD_311_Housing.csv')
if not os.path.exists(datapath_combineddata):
    combined_sfpd_311_sales.to_csv(datapath_combineddata, index=False)